In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import History 

In [8]:
option_data = pd.read_csv("BlackScholes/bs_1_parameters.csv", index_col=0)
option_data = option_data[option_data.price > 1.0]
X = option_data[option_data.columns[0:4]].values
Y = option_data[option_data.columns[-1]].values
option_data

,r,sigma,maturity,moneyness,price
0,0.084894,0.017301,1.0,0.500000,54.069514
1,0.084894,0.017301,1.0,0.541667,50.241973
2,0.084894,0.017301,1.0,0.583333,46.414433
3,0.084894,0.017301,1.0,0.625000,42.586892
4,0.084894,0.017301,1.0,0.666667,38.759352
...,...,...,...,...,...
624995,0.058298,0.131999,3.0,1.333333,4.818583
624996,0.058298,0.131999,3.0,1.375000,3.944366
624997,0.058298,0.131999,3.0,1.416667,3.210616
624998,0.058298,0.131999,3.0,1.458333,2.599606


In [9]:
print(X)

[[0.08489392 0.01730083 1.         0.5       ]
 [0.08489392 0.01730083 1.         0.54166667]
 [0.08489392 0.01730083 1.         0.58333333]
 ...
 [0.05829797 0.13199939 3.         1.41666667]
 [0.05829797 0.13199939 3.         1.45833333]
 [0.05829797 0.13199939 3.         1.5       ]]


In [10]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(X.shape[1], activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanAbsoluteError()
)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=1)

In [12]:
history = model.fit(X_train, Y_train, batch_size=16, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/100
6841/6844 [============================>.] - ETA: 0s - loss: 2.0234

KeyboardInterrupt: 

In [ ]:
Y = model.predict(X)

275/275 [==============================] - 1s 2ms/step


In [ ]:
option_data["NN price"] = Y
option_data["Error"] = 100 * np.abs(option_data["price"] - option_data["NN price"]) / option_data["price"]
option_data = option_data.sort_values(by=["Error"], )
option_data

,Unnamed: 0,r,sigma,maturity,moneyness,price,NN price,Error
7977,7977,0.035842,0.169959,2.555556,1.277778,5.210140,5.209159,0.018838
2,2,0.012053,0.070812,1.000000,0.722222,28.643079,28.636126,0.024277
4987,4987,0.137674,0.078809,2.777778,1.277778,13.764711,13.760055,0.033829
1287,1287,0.132056,0.104667,2.777778,1.277778,13.808245,13.803176,0.036710
4386,4386,0.098129,0.243498,2.777778,1.166667,21.398399,21.389015,0.043855
...,...,...,...,...,...,...,...,...
9535,9535,0.024598,0.030919,1.666667,1.055556,1.030128,7.975815,674.254899
9555,9555,0.024598,0.030919,2.111111,1.055556,1.688978,13.131074,677.456619
8686,8686,0.034883,0.053484,2.777778,1.166667,1.442735,11.903559,725.068699
9545,9545,0.024598,0.030919,1.888889,1.055556,1.347101,11.466783,751.218967
